In [1]:
from keras.layers import Conv2D, Conv2DTranspose, Flatten, Input, Dense, Reshape
from keras.datasets import mnist
from keras import Model
import numpy as np
import matplotlib.pyplot as plt 
import matplotlib.image as mpimg
import glob
%matplotlib inline

/Users/timur89833239972/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [3]:
%%time 
img_size = 64

samples = glob.glob('./lfwcrop_gray/faces/*')
data = []

for s in samples:
    img = mpimg.imread(s)
    data.append(np.expand_dims(img, 0))
        
# faces = np.concatenate(faces)

CPU times: user 9.18 s, sys: 12.6 s, total: 21.7 s
Wall time: 1min 22s


In [4]:
faces = np.concatenate(data, axis=0)

In [5]:
faces.shape

(13233, 64, 64)

In [6]:
faces = np.expand_dims(faces, -1)

In [7]:
# prepare data
faces = faces / 255.

#### Fit autoencoder

In [32]:
# encoder
input_ = Input((64, 64, 1)) # 64
x = Conv2D(filters=8, kernel_size=2, strides=2, activation='relu')(input_) # 32
x = Conv2D(filters=16, kernel_size=2, strides=2, activation='relu')(x) # 16
x = Conv2D(filters=32, kernel_size=2, strides=2, activation='relu')(x) # 8
x = Conv2D(filters=64, kernel_size=2, strides=2, activation='relu')(x) # 4
x = Conv2D(filters=128, kernel_size=2, strides=2, activation='relu')(x) # 2
flat = Flatten()(x)
latent = Dense(128)(flat)

# decoder
reshape = Reshape((2,2,32)) #2
conv_2t_1 = Conv2DTranspose(filters=128, kernel_size=2, strides=2, activation='relu') # 4
conv_2t_2 = Conv2DTranspose(filters=64, kernel_size=2, strides=2, activation='relu') # 8
conv_2t_3 = Conv2DTranspose(filters=32, kernel_size=2, strides=2, activation='relu') # 16
conv_2t_4 = Conv2DTranspose(filters=16, kernel_size=2, strides=2, activation='relu') # 32
conv_2t_5 = Conv2DTranspose(filters=1, kernel_size=2, strides=2, activation='sigmoid') # 64

x = reshape(latent)
x = conv_2t_1(x)
x = conv_2t_2(x)
x = conv_2t_3(x)
x = conv_2t_4(x)
decoded = conv_2t_5(x) # 64

autoencoder = Model(input_, decoded)
encoder = Model(input_, latent)

decoder_input = Input((128,))
x_ = reshape(decoder_input)
x_ = conv_2t_1(x_)
x_ = conv_2t_2(x_)
x_ = conv_2t_3(x_)
x_ = conv_2t_4(x_)
decoded_ = conv_2t_5(x_) # 64
decoder = Model(decoder_input, decoded_)

In [33]:
autoencoder.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, 64, 64, 1)         0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 32, 32, 8)         40        
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 16, 16, 16)        528       
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 8, 8, 32)          2080      
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 4, 4, 64)          8256      
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 2, 2, 128)         32896     
_________________________________________________________________
flatten_2 (Flatten)          (None, 512)               0         
__________

In [34]:
autoencoder.compile(optimizer='adam', loss='binary_crossentropy')

In [35]:
# увеличь кол-во эпох дома!

autoencoder.fit(faces, faces, epochs=3)

Epoch 1/3
13233/13233 [==============================] - 14s 1ms/step - loss: 0.6598
Epoch 2/3
13233/13233 [==============================] - 13s 998us/step - loss: 0.6415
Epoch 3/3
13233/13233 [==============================] - 13s 1ms/step - loss: 0.6394


In [36]:
latents = encoder.predict(faces[:10])

In [37]:
latents.shape

(10, 128)

In [ ]:
latents_for_our_faces = encoder.predict(НАШИ КРОПЫ ИЗ ИЗОБРАЖЕНИЯ) # 3, 128
latents_for_BOKTA = encoder.predict(ФОТО БОКТЫ) # 1, 128